# Programmatic example with MARS/GRIB input

In [ ]:
%%javascript
// leave this in to disable autoscroll in Jupyter notebook
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Imports

In [ ]:
import os
from pathlib import Path
import json
import numpy as np
import fcpy

## Load data

In [ ]:
if (Path().home() / ".ecmwfapirc").exists():
    fpath = "../samples/cases/hplp_ml_q.json"
    request = json.load(open(fpath))["mars"]
    ds = fcpy.fetch_mars_dataset(request)
else:
    ds = fcpy.open_dataset("../data/mars.grib")
    
ds = ds.isel(hybrid=[0, 1])
ds

## Define and run experiment

In [ ]:
suite = fcpy.Suite(
    ds,
    baseline=fcpy.Float(bits=32),
    compressors=[fcpy.LinQuantization(), fcpy.Round()],
    metrics=[fcpy.Difference, fcpy.AbsoluteError],
    bits=[13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
    max_chunk_size_bytes=451 * 900 * 6,
    skip_histograms=True,
)

## Lineplot: evaluate the effect of bits

In [ ]:
suite.lineplot(fcpy.AbsoluteError, reduction="max")

## Boxplot: evaluate statistical characteristics of errors

In [ ]:
bits = 13
suite.boxplot(metric=fcpy.Difference, compressor="LinQuantization", bits=bits)
suite.boxplot(metric=fcpy.Difference, compressor="Round", bits=bits)

## Spatial plots: evaluate the spatial distribution of errors

In [ ]:
if not os.getenv("SPHINX_BUILD"):
    # Here we compare the spatial change of specific humidity
    # by rounding values to 8 bits
    fcpy.spatialplot(
        ds=ds,
        baseline=fcpy.Float(bits=32),
        var_name="q",
        compressor=fcpy.Round(bits=6),
        metric=fcpy.Difference,
        latitude=0,
        longitude=0,
        hybrid=1,
        step=np.timedelta64(0, "ns"),
        third_dim="hybrid",
    )